## Download dataset

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!unzip 'drive/MyDrive/emg_imu_data.zip'


## Fix Random Seed

In [4]:
import numpy as np
import torch
import random

def set_seed(seed):
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

set_seed(87)

# Data

## Dataset
- Original dataset is [Voxceleb2](https://www.robots.ox.ac.uk/~vgg/data/voxceleb/vox2.html).
- The [license](https://creativecommons.org/licenses/by/4.0/) and [complete version](https://www.robots.ox.ac.uk/~vgg/data/voxceleb/files/license.txt) of Voxceleb2.
- We randomly select 600 speakers from Voxceleb2.
- Then preprocess the raw waveforms into mel-spectrograms.

- Args:
  - data_dir: The path to the data directory.
  - metadata_path: The path to the metadata.
  - segment_len: The length of audio segment for training.
- The architecture of data directory \\
  - data directory \\
  |---- metadata.json \\
  |---- testdata.json \\
  |---- mapping.json \\
  |---- uttr-{random string}.pt \\

- The information in metadata
  - "n_mels": The dimention of mel-spectrogram.
  - "speakers": A dictionary.
    - Key: speaker ids.
    - value: "feature_path" and "mel_len"


For efficiency, we segment the mel-spectrograms into segments in the traing step.

In [18]:
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import StandardScaler, LabelEncoder
import numpy as np
import os

class EMGDataset(Dataset):
    def __init__(self, directory, seq_length=790, scaler=None):
        self.directory = directory
        self.seq_length = seq_length
        self.files = []
        self.labels = []
        self.scaler = scaler  # 将预先计算好的Scaler传入

        for f in os.listdir(directory):
            if f.endswith("_emg.txt"):
                filepath = os.path.join(directory, f)
                if os.path.getsize(filepath) > 0:
                    self.files.append(filepath)
                    self.labels.append(f.split('_')[0])

        self.label_encoder = LabelEncoder()
        self.labels = self.label_encoder.fit_transform(self.labels)
        self.labels = torch.from_numpy(self.labels).long()

        self.num_classes=len(np.unique(self.labels))

    def __len__(self):
        return len(self.files)

    def __getitem__(self, idx):
        filepath = self.files[idx]
        emg_signals = np.loadtxt(filepath)
        if emg_signals.shape[0] < self.seq_length:
            pad = np.zeros((self.seq_length - emg_signals.shape[0], emg_signals.shape[1]))
            emg_signals = np.vstack([emg_signals, pad])
        elif emg_signals.shape[0] > self.seq_length:
            emg_signals = emg_signals[:self.seq_length, :]

        if self.scaler:
            emg_signals = self.scaler.transform(emg_signals)  # 使用预先拟合的Scaler进行转换

        emg_signals = torch.from_numpy(emg_signals).float()
        return emg_signals, self.labels[idx]

    def get_num_classes(self):
		    return self.num_classes



# # Assuming 'path_to_your_directory' is the path to your data
# dataset = EMGDataset('path_to_your_directory')
# dataloader = DataLoader(dataset, batch_size=32, shuffle=True)


## 计算sacler

In [3]:
import numpy as np
import os

def incremental_mean_std(directory, seq_length=790):
    mean = np.zeros((seq_length, 8))  # 假设每个样本有8个特征
    M2 = np.zeros((seq_length, 8))
    n = 0
    for filename in os.listdir(directory):
        if filename.endswith("_emg.txt"):
            filepath = os.path.join(directory, filename)
            if os.path.getsize(filepath) > 0:
                data = np.loadtxt(filepath)
                if data.size == 0:  # 检查数据是否为空
                    continue
                if data.shape[0] < seq_length:
                    # 填充不足的数据
                    pad = np.zeros((seq_length - data.shape[0], data.shape[1]))
                    data = np.vstack([data, pad])
                elif data.shape[0] > seq_length:
                    # 截断超出的数据
                    data = data[:seq_length, :]
                n += 1
                delta = data - mean
                mean += delta / n
                M2 += delta * (data - mean)

    std = np.sqrt(M2 / n) if n > 1 else np.sqrt(M2)
    return mean, std

# directory = 'emg_data'
# mean, std = incremental_mean_std(directory)

# 用得到的 mean 和 std 来创建一个 StandardScaler-like 的类
class CustomScaler:
    def __init__(self, mean, std):
        self.mean = mean
        self.std = std

    def transform(self, X):
        return (X - self.mean) / self.std


In [4]:
directory = 'emg_data'
mean, std = incremental_mean_std(directory)
scaler = CustomScaler(mean, std)

In [8]:
import pickle

# 从文件加载scaler对象
with open('drive/MyDrive/scaler.pkl', 'rb') as f:
    loaded_scaler = pickle.load(f)

    scaler = loaded_scaler

In [ ]:
# 使用加载的scaler来转换数据
# transformed_data = loaded_scaler.transform(new_data)

In [8]:
dataset = EMGDataset(directory=directory, scaler=scaler)

In [ ]:
directory='emg_data'
# scaler = compute_statistics(directory)

dataset = EMGDataset(directory=directory, scaler=scaler)
# dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

## 确定seq_length
确定seq_length

In [ ]:
import numpy as np
import os

lengths0 = []
lengths1 = []
def analyze_emg_lengths(directory):

    for filename in os.listdir(directory):
        if filename.endswith("_emg.txt"):
            filepath = os.path.join(directory, filename)
            try:
                emg_signals = np.loadtxt(filepath)
                if emg_signals.shape[0]!=0:
                  lengths0.append(emg_signals.shape[0])
                  lengths1.append(emg_signals.shape[1])

            except:
                # Handling empty or corrupted files
                print(f"Could not load data from {filename}")
    return lengths0, lengths1

# Assuming 'path_to_your_directory' is the path to your data
# directory = 'emg_data'
# lengths = analyze_emg_lengths(directory)

# # Calculate statistics
# if lengths0:
#     min_length0 = np.min(lengths0)
#     max_length0 = np.max(lengths0)
#     avg_length0 = np.mean(lengths0)
#     median_length0 = np.median(lengths0)

#     min_length1 = np.min(lengths1)
#     max_length1 = np.max(lengths1)
#     avg_length1 = np.mean(lengths1)
#     median_length1 = np.median(lengths1)

#     print(f"Minimum length: {min_length0}, {min_length1}")
#     print(f"Maximum length: {max_length0}， {max_length1}")
#     print(f"Average length: {avg_length0:.2f}, {avg_length1:.2f}")
#     print(f"Median length: {median_length0}, {median_length1}")
# else:
#     print("No data available to analyze.")


## Dataloader
- Split dataset into training dataset(90%) and validation dataset(10%).
- Create dataloader to iterate the data.

In [21]:
# scaler=scaler
scaler=None

In [19]:
import torch
from torch.utils.data import DataLoader, random_split
from torch.nn.utils.rnn import pad_sequence

def emg_collate_batch(batch):
    data, labels = zip(*batch)
    return torch.stack(data), torch.tensor(labels)

def get_emg_dataloader(data_dir, batch_size, n_workers):
    dataset = EMGDataset(directory=data_dir, scaler=scaler)
    num_classes = dataset.get_num_classes()
    # 分割数据集为训练集和验证集
    train_len = int(0.9 * len(dataset))
    lengths = [train_len, len(dataset) - train_len]
    train_set, valid_set = random_split(dataset, lengths)

    train_loader = DataLoader(
        train_set,
        batch_size=batch_size,
        shuffle=True,
        drop_last=True,
        num_workers=n_workers,
        pin_memory=True,
        collate_fn=emg_collate_batch,
    )
    valid_loader = DataLoader(
        valid_set,
        batch_size=batch_size,
        num_workers=n_workers,
        drop_last=True,
        pin_memory=True,
        collate_fn=emg_collate_batch,
    )

    return train_loader, valid_loader, num_classes


# Model
- TransformerEncoderLayer:
  - Base transformer encoder layer in [Attention Is All You Need](https://arxiv.org/abs/1706.03762)
  - Parameters:
    - d_model: the number of expected features of the input (required).

    - nhead: the number of heads of the multiheadattention models (required).

    - dim_feedforward: the dimension of the feedforward network model (default=2048).

    - dropout: the dropout value (default=0.1).

    - activation: the activation function of intermediate layer, relu or gelu (default=relu).

- TransformerEncoder:
  - TransformerEncoder is a stack of N transformer encoder layers
  - Parameters:
    - encoder_layer: an instance of the TransformerEncoderLayer() class (required).

    - num_layers: the number of sub-encoder-layers in the encoder (required).

    - norm: the layer normalization component (optional).

In [13]:
import torch.nn as nn

class TransformerModel(nn.Module):
    def __init__(self, input_dim, num_heads, ff_dim, num_classes):
        super(TransformerModel, self).__init__()
        self.encoder_layer = nn.TransformerEncoderLayer(d_model=input_dim, nhead=num_heads, dim_feedforward=ff_dim)
        self.encoder_layer = nn.TransformerEncoderLayer(d_model=input_dim, nhead=num_heads, dim_feedforward=ff_dim, batch_first=True)
        # self.encoder_layer = nn.TransformerEncoderLayer(d_model=input_dim, nhead=num_heads, dim_feedforward=input_dim*2)
        self.encoder = nn.TransformerEncoder(self.encoder_layer, num_layers=1)
        self.pool = nn.AdaptiveAvgPool1d(1)
        self.fc = nn.Linear(input_dim, num_classes)

    def forward(self, x):
        x = x.permute(1, 0, 2)  # PyTorch transformer expects input as (seq_len, batch, features)
        x = self.encoder(x)
        x = x.permute(1, 2, 0)  # Revert to (batch, features, seq_len) for pooling
        x = self.pool(x).squeeze(-1)
        x = self.fc(x)
        return x

# Assuming 8 features per timestep and the sequence length is the same for all samples
# model = TransformerModel(input_dim=8, num_heads=2, ff_dim=256, num_classes=len(np.unique(dataset.labels)))


In [ ]:
# import torch
# import torch.nn as nn
# import torch.nn.functional as F


# class Classifier(nn.Module):
# 	def __init__(self, d_model=224, n_spks=600, dropout=0.1):
# 		super().__init__()
# 		# Project the dimension of features from that of input into d_model.
# 		self.prenet = nn.Linear(40, d_model)
# 		# TODO:
# 		#   Change Transformer to Conformer.
# 		#   https://arxiv.org/abs/2005.08100
# 		self.encoder_layer = nn.TransformerEncoderLayer(
# 			d_model=d_model, dim_feedforward=d_model*2, nhead=2, dropout=dropout
# 		)
# 		self.encoder = nn.TransformerEncoder(self.encoder_layer, num_layers=3)

# 		# Project the the dimension of features from d_model into speaker nums.
# 		self.pred_layer = nn.Sequential(
# 			nn.BatchNorm1d(d_model),
# 			nn.Linear(d_model, n_spks),
# 		)

# 	def forward(self, mels):
# 		"""
# 		args:
# 			mels: (batch size, length, 40)
# 		return:
# 			out: (batch size, n_spks)
# 		"""
# 		# out: (batch size, length, d_model)
# 		out = self.prenet(mels)
# 		# out: (length, batch size, d_model)
# 		out = out.permute(1, 0, 2)
# 		# The encoder layer expect features in the shape of (length, batch size, d_model).
# 		out = self.encoder_layer(out)
# 		# out: (batch size, length, d_model)
# 		out = out.transpose(0, 1)
# 		# mean pooling
# 		stats = out.mean(dim=1)

# 		# out: (batch, n_spks)
# 		out = self.pred_layer(stats)
# 		return out

# Learning rate schedule
- For transformer architecture, the design of learning rate schedule is different from that of CNN.
- Previous works show that the warmup of learning rate is useful for training models with transformer architectures.
- The warmup schedule
  - Set learning rate to 0 in the beginning.
  - The learning rate increases linearly from 0 to initial learning rate during warmup period.

In [14]:
import math

import torch
from torch.optim import Optimizer
from torch.optim.lr_scheduler import LambdaLR


def get_cosine_schedule_with_warmup(
	optimizer: Optimizer,
	num_warmup_steps: int,
	num_training_steps: int,
	num_cycles: float = 0.5,
	last_epoch: int = -1,
):
	"""
	Create a schedule with a learning rate that decreases following the values of the cosine function between the
	initial lr set in the optimizer to 0, after a warmup period during which it increases linearly between 0 and the
	initial lr set in the optimizer.

	Args:
		optimizer (:class:`~torch.optim.Optimizer`):
		The optimizer for which to schedule the learning rate.
		num_warmup_steps (:obj:`int`):
		The number of steps for the warmup phase.
		num_training_steps (:obj:`int`):
		The total number of training steps.
		num_cycles (:obj:`float`, `optional`, defaults to 0.5):
		The number of waves in the cosine schedule (the defaults is to just decrease from the max value to 0
		following a half-cosine).
		last_epoch (:obj:`int`, `optional`, defaults to -1):
		The index of the last epoch when resuming training.

	Return:
		:obj:`torch.optim.lr_scheduler.LambdaLR` with the appropriate schedule.
	"""
	def lr_lambda(current_step):
		# Warmup
		if current_step < num_warmup_steps:
			return float(current_step) / float(max(1, num_warmup_steps))
		# decadence
		progress = float(current_step - num_warmup_steps) / float(
			max(1, num_training_steps - num_warmup_steps)
		)
		return max(
			0.0, 0.5 * (1.0 + math.cos(math.pi * float(num_cycles) * 2.0 * progress))
		)

	return LambdaLR(optimizer, lr_lambda, last_epoch)

# Model Function
- Model forward function.

In [15]:
import torch

def model_fn(batch, model, criterion, device):
    """Forward a batch through the model."""

    # 解包批数据
    data, labels = batch
    # 将数据和标签转移到指定设备（例如 GPU）
    data = data.to(device)
    labels = labels.to(device)

    # 通过模型进行前向传播得到输出
    outs = model(data)

    # 计算损失
    loss = criterion(outs, labels)

    # 计算预测的类别（最高得分的类别）
    preds = outs.argmax(1)
    # 计算准确率
    accuracy = torch.mean((preds == labels).float())

    return loss, accuracy


# Validate
- Calculate accuracy of the validation set.

In [16]:
from tqdm import tqdm
import torch

def valid(dataloader, model, criterion, device):
    """Validate on validation set."""
    model.eval()
    running_loss = 0.0
    running_accuracy = 0.0
    pbar = tqdm(total=len(dataloader.dataset), ncols=0, desc="Valid", unit="sample")

    for i, batch in enumerate(dataloader):
        with torch.no_grad():
            loss, accuracy = model_fn(batch, model, criterion, device)
            running_loss += loss.item()
            running_accuracy += accuracy.item()

        pbar.update(dataloader.batch_size)
        pbar.set_postfix(
            loss=f"{running_loss / (i+1):.2f}",
            accuracy=f"{running_accuracy / (i+1):.2f}",
        )

    pbar.close()
    model.train()

    return running_accuracy / len(dataloader)


# Main function

In [22]:
from tqdm import tqdm
import torch
import torch.nn as nn
from torch.optim import AdamW
from torch.utils.data import DataLoader

def parse_args():
    """arguments"""
    config = {
        "data_dir": "./emg_data",
        "save_path": "model.ckpt",
        "batch_size": 32,
        "n_workers": 2,
        "valid_steps": 500,
        "warmup_steps": 500,
        "save_steps": 2500,
        "total_steps": 7500,
    }
    return config

def main(
    data_dir,
    save_path,
    batch_size,
    n_workers,
    valid_steps,
    warmup_steps,
    total_steps,
    save_steps,
):
    """Main function."""
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"[Info]: Use {device} now!")

    train_loader, valid_loader, num_classes = get_emg_dataloader(data_dir, batch_size, n_workers)
    train_iterator = iter(train_loader)
    print(f"[Info]: Finish loading data!", flush=True)

    model = TransformerModel(input_dim=8, num_heads=2, ff_dim=256, num_classes=num_classes).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = AdamW(model.parameters(), lr=1e-3)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=total_steps)

    print(f"[Info]: Finish creating model!", flush=True)

    best_accuracy = -1.0
    best_state_dict = None

    pbar = tqdm(total=valid_steps, ncols=0, desc="Train", unit=" step")

    for step in range(total_steps):
        try:
            batch = next(train_iterator)
        except StopIteration:
            train_iterator = iter(train_loader)
            batch = next(train_iterator)

        loss, accuracy = model_fn(batch, model, criterion, device)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        scheduler.step()

        pbar.update()
        pbar.set_postfix(
            loss=f"{loss.item():.2f}",
            accuracy=f"{accuracy:.2f}",
            step=step + 1,
        )

        if (step + 1) % valid_steps == 0:
            pbar.close()
            valid_accuracy = valid(valid_loader, model, criterion, device)
            if valid_accuracy > best_accuracy:
                best_accuracy = valid_accuracy
                best_state_dict = model.state_dict()

            pbar = tqdm(total=valid_steps, ncols=0, desc="Train", unit=" step")

        if (step + 1) % save_steps == 0 and best_state_dict is not None:
            torch.save(best_state_dict, save_path)
            print(f"Step {step + 1}, best model saved. (accuracy={best_accuracy:.4f})")

    pbar.close()

if __name__ == "__main__":
    main(**parse_args())


[Info]: Use cuda now!


/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


[Info]: Finish loading data!
[Info]: Finish creating model!


Train: 100% 500/500 [00:25<00:00, 19.61 step/s, accuracy=0.00, loss=6.07, step=500]
Valid: 100% 7616/7622 [00:06<00:00, 1098.77sample/s, accuracy=0.01, loss=6.03]
Train: 100% 500/500 [00:24<00:00, 20.62 step/s, accuracy=0.03, loss=6.02, step=1000]
Valid: 100% 7616/7622 [00:06<00:00, 1107.92sample/s, accuracy=0.01, loss=5.98]
Train: 100% 500/500 [00:24<00:00, 20.54 step/s, accuracy=0.03, loss=5.74, step=1500]
Valid: 100% 7616/7622 [00:06<00:00, 1095.29sample/s, accuracy=0.01, loss=5.93]
Train: 100% 500/500 [00:23<00:00, 21.26 step/s, accuracy=0.03, loss=5.87, step=2000]
Valid: 100% 7616/7622 [00:06<00:00, 1134.44sample/s, accuracy=0.02, loss=5.88]
Train: 100% 500/500 [00:24<00:00, 20.36 step/s, accuracy=0.00, loss=5.77, step=2500]
Valid: 100% 7616/7622 [00:12<00:00, 616.31sample/s, accuracy=0.02, loss=5.83]
Train:   1% 6/500 [00:00<00:15, 31.80 step/s, accuracy=0.03, loss=5.93, step=2506]

Step 2500, best model saved. (accuracy=0.0183)


Train: 100% 500/500 [00:24<00:00, 20.43 step/s, accuracy=0.03, loss=6.13, step=3000]
Valid: 100% 7616/7622 [00:08<00:00, 863.48sample/s, accuracy=0.02, loss=5.80] 
Train: 100% 500/500 [00:22<00:00, 22.13 step/s, accuracy=0.03, loss=5.62, step=3500]
Valid: 100% 7616/7622 [00:09<00:00, 843.51sample/s, accuracy=0.02, loss=5.77] 
Train: 100% 500/500 [00:21<00:00, 23.04 step/s, accuracy=0.00, loss=5.98, step=4000]
Valid: 100% 7616/7622 [00:08<00:00, 868.44sample/s, accuracy=0.02, loss=5.74] 
Train: 100% 500/500 [00:22<00:00, 22.43 step/s, accuracy=0.00, loss=5.51, step=4500]
Valid: 100% 7616/7622 [00:08<00:00, 900.82sample/s, accuracy=0.02, loss=5.73] 
Train: 100% 500/500 [00:23<00:00, 21.40 step/s, accuracy=0.00, loss=6.02, step=5000]
Valid: 100% 7616/7622 [00:08<00:00, 875.19sample/s, accuracy=0.02, loss=5.71] 
Train:   2% 8/500 [00:00<00:09, 54.08 step/s, accuracy=0.00, loss=5.92, step=5008]

Step 5000, best model saved. (accuracy=0.0202)


Train: 100% 500/500 [00:22<00:00, 22.57 step/s, accuracy=0.06, loss=5.74, step=5500]
Valid: 100% 7616/7622 [00:08<00:00, 882.27sample/s, accuracy=0.02, loss=5.70] 
Train:  70% 351/500 [00:14<00:08, 18.51 step/s, accuracy=0.00, loss=5.53, step=5851]

KeyboardInterrupt: 

## 示例

In [ ]:
from tqdm import tqdm

import torch
import torch.nn as nn
from torch.optim import AdamW
from torch.utils.data import DataLoader, random_split


def parse_args():
	"""arguments"""
	config = {
		"data_dir": "./Dataset",
		"save_path": "model.ckpt",
		"batch_size": 32,
		"n_workers": 8,
		"valid_steps": 2000,
		"warmup_steps": 1000,
		"save_steps": 10000,
		"total_steps": 100000,
	}

	return config


def main(
	data_dir,
	save_path,
	batch_size,
	n_workers,
	valid_steps,
	warmup_steps,
	total_steps,
	save_steps,
):
	"""Main function."""
	device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
	print(f"[Info]: Use {device} now!")

	train_loader, valid_loader, speaker_num = get_dataloader(data_dir, batch_size, n_workers)
	train_iterator = iter(train_loader)
	print(f"[Info]: Finish loading data!",flush = True)

	model = Classifier(n_spks=speaker_num).to(device)
	criterion = nn.CrossEntropyLoss()
	optimizer = AdamW(model.parameters(), lr=1e-3)
	scheduler = get_cosine_schedule_with_warmup(optimizer, warmup_steps, total_steps)
	print(f"[Info]: Finish creating model!",flush = True)

	best_accuracy = -1.0
	best_state_dict = None

	pbar = tqdm(total=valid_steps, ncols=0, desc="Train", unit=" step")

	for step in range(total_steps):
		# Get data
		try:
			batch = next(train_iterator)
		except StopIteration:
			train_iterator = iter(train_loader)
			batch = next(train_iterator)

		loss, accuracy = model_fn(batch, model, criterion, device)
		batch_loss = loss.item()
		batch_accuracy = accuracy.item()

		# Updata model
		loss.backward()
		optimizer.step()
		scheduler.step()
		optimizer.zero_grad()

		# Log
		pbar.update()
		pbar.set_postfix(
			loss=f"{batch_loss:.2f}",
			accuracy=f"{batch_accuracy:.2f}",
			step=step + 1,
		)

		# Do validation
		if (step + 1) % valid_steps == 0:
			pbar.close()

			valid_accuracy = valid(valid_loader, model, criterion, device)

			# keep the best model
			if valid_accuracy > best_accuracy:
				best_accuracy = valid_accuracy
				best_state_dict = model.state_dict()

			pbar = tqdm(total=valid_steps, ncols=0, desc="Train", unit=" step")

		# Save the best model so far.
		if (step + 1) % save_steps == 0 and best_state_dict is not None:
			torch.save(best_state_dict, save_path)
			pbar.write(f"Step {step + 1}, best model saved. (accuracy={best_accuracy:.4f})")

	pbar.close()


if __name__ == "__main__":
	main(**parse_args())

[Info]: Use cuda now!


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


[Info]: Finish loading data!
[Info]: Finish creating model!


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/transformer.py:286: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")
Train:  80% 1593/2000 [00:36<00:06, 64.73 step/s, accuracy=0.31, loss=3.69, step=1593]/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
Train: 100% 2000/2000 [00:44<00:00, 45.04 step/s, accuracy=0.41, loss=2.86, step=2000]
Valid: 100% 5664/5667 [00:03<00:00, 1561.83 uttr/s, accuracy=0.34, loss=3.13]
Train: 100% 2000/2000 [00:49<00:00, 40.44 step/s, accuracy=0.38, loss=3.04, step=4000]
Valid: 100% 5664/5667 [00:04<00:00, 1138.34 

Step 10000, best model saved. (accuracy=0.5922)


...

Step 80000, best model saved. (accuracy=0.8385)


Train: 100% 2000/2000 [00:44<00:00, 44.97 step/s, accuracy=0.97, loss=0.16, step=82000]
Valid: 100% 5664/5667 [00:03<00:00, 1741.05 uttr/s, accuracy=0.84, loss=0.74]
Train: 100% 2000/2000 [00:47<00:00, 41.67 step/s, accuracy=0.97, loss=0.13, step=84000]
Valid: 100% 5664/5667 [00:04<00:00, 1409.69 uttr/s, accuracy=0.84, loss=0.75]
Train: 100% 2000/2000 [00:43<00:00, 45.81 step/s, accuracy=0.97, loss=0.12, step=86000]
Valid: 100% 5664/5667 [00:02<00:00, 1911.13 uttr/s, accuracy=0.84, loss=0.71]
Train: 100% 2000/2000 [00:44<00:00, 44.63 step/s, accuracy=1.00, loss=0.16, step=88000]
Valid: 100% 5664/5667 [00:03<00:00, 1860.13 uttr/s, accuracy=0.84, loss=0.75]
Train: 100% 2000/2000 [00:43<00:00, 46.27 step/s, accuracy=1.00, loss=0.16, step=9e+4]
Valid: 100% 5664/5667 [00:04<00:00, 1331.91 uttr/s, accuracy=0.85, loss=0.72]
Train:   0% 5/2000 [00:00<01:10, 28.41 step/s, accuracy=0.94, loss=0.25, step=9e+4]

Step 90000, best model saved. (accuracy=0.8501)


Train: 100% 2000/2000 [00:43<00:00, 46.41 step/s, accuracy=0.94, loss=0.20, step=92000]
Valid: 100% 5664/5667 [00:04<00:00, 1176.20 uttr/s, accuracy=0.85, loss=0.73]
Train: 100% 2000/2000 [00:44<00:00, 45.32 step/s, accuracy=0.94, loss=0.08, step=94000]
Valid: 100% 5664/5667 [00:04<00:00, 1139.50 uttr/s, accuracy=0.85, loss=0.72]
Train: 100% 2000/2000 [00:47<00:00, 42.17 step/s, accuracy=0.91, loss=0.18, step=96000]
Valid: 100% 5664/5667 [00:02<00:00, 1915.03 uttr/s, accuracy=0.85, loss=0.73]
Train: 100% 2000/2000 [00:43<00:00, 45.76 step/s, accuracy=0.97, loss=0.12, step=98000]
Valid: 100% 5664/5667 [00:02<00:00, 1905.56 uttr/s, accuracy=0.84, loss=0.73]
Train: 100% 2000/2000 [00:43<00:00, 45.79 step/s, accuracy=0.97, loss=0.13, step=1e+5]
Valid: 100% 5664/5667 [00:03<00:00, 1835.94 uttr/s, accuracy=0.85, loss=0.71]
Train:   0% 0/2000 [00:00<?, ? step/s]


Step 100000, best model saved. (accuracy=0.8520)


# Inference

## Dataset of inference

In [ ]:
import os
import json
import torch
from pathlib import Path
from torch.utils.data import Dataset


class InferenceDataset(Dataset):
	def __init__(self, data_dir):
		testdata_path = Path(data_dir) / "testdata.json"
		metadata = json.load(testdata_path.open())
		self.data_dir = data_dir
		self.data = metadata["utterances"]

	def __len__(self):
		return len(self.data)

	def __getitem__(self, index):
		utterance = self.data[index]
		feat_path = utterance["feature_path"]
		mel = torch.load(os.path.join(self.data_dir, feat_path))

		return feat_path, mel


def inference_collate_batch(batch):
	"""Collate a batch of data."""
	feat_paths, mels = zip(*batch)

	return feat_paths, torch.stack(mels)

## Main funcrion of Inference

In [ ]:
import json
import csv
from pathlib import Path
from tqdm.notebook import tqdm

import torch
from torch.utils.data import DataLoader

def parse_args():
	"""arguments"""
	config = {
		"data_dir": "./Dataset",
		"model_path": "./model.ckpt",
		"output_path": "./output.csv",
	}

	return config


def main(
	data_dir,
	model_path,
	output_path,
):
	"""Main function."""
	device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
	print(f"[Info]: Use {device} now!")

	mapping_path = Path(data_dir) / "mapping.json"
	mapping = json.load(mapping_path.open())

	dataset = InferenceDataset(data_dir)
	dataloader = DataLoader(
		dataset,
		batch_size=1,
		shuffle=False,
		drop_last=False,
		num_workers=8,
		collate_fn=inference_collate_batch,
	)
	print(f"[Info]: Finish loading data!",flush = True)

	speaker_num = len(mapping["id2speaker"])
	model = Classifier(n_spks=speaker_num).to(device)
	model.load_state_dict(torch.load(model_path))
	model.eval()
	print(f"[Info]: Finish creating model!",flush = True)

	results = [["Id", "Category"]]
	for feat_paths, mels in tqdm(dataloader):
		with torch.no_grad():
			mels = mels.to(device)
			outs = model(mels)
			preds = outs.argmax(1).cpu().numpy()
			for feat_path, pred in zip(feat_paths, preds):
				results.append([feat_path, mapping["id2speaker"][str(pred)]])

	with open(output_path, 'w', newline='') as csvfile:
		writer = csv.writer(csvfile)
		writer.writerows(results)


if __name__ == "__main__":
	main(**parse_args())

[Info]: Use cuda now!
[Info]: Finish loading data!
[Info]: Finish creating model!


  0%|          | 0/8000 [00:00<?, ?it/s]